# SVD Hyperparameter Tuning

This notebook demonstrates how to tune the hyperparameters of the SVD algorithm using GridSearchCV and compares the performance of tuned vs untuned SVD against random recommendations.

### Setup and Imports

First, let's import the necessary libraries and set up our random seeds for reproducibility.

In [1]:
from recsys.MovieLens import MovieLens
from surprise import SVD
from surprise import NormalPredictor
from recsys.Evaluator import Evaluator
from surprise.model_selection import GridSearchCV

import random
import numpy as np

np.random.seed(0)
random.seed(0)

### Load Data

Load the MovieLens dataset and prepare it for evaluation.

In [ ]:
lens, ratings_data, rankings = MovieLens.load()

### Grid Search for Best Parameters

We'll use GridSearchCV to find the optimal hyperparameters for the SVD algorithm. We'll search over:
- Number of epochs (20, 30)
- Learning rate (0.005, 0.010)
- Number of factors (50, 100)

In [ ]:
print("Searching for best parameters...")
param_grid = {
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.010],
    'n_factors': [50, 100]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(ratings_data)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print("Best parameters:", gs.best_params['rmse'])

### Initialize Evaluator and Add Algorithms

Now we'll create an evaluator and add three algorithms to compare:
1. SVD with tuned parameters
2. SVD with default parameters
3. Random recommendations (baseline)

In [ ]:
evaluator = Evaluator(ratings_data, rankings)

# Add tuned SVD
params = gs.best_params['rmse']
svd_tuned = SVD(n_epochs=params['n_epochs'], 
               lr_all=params['lr_all'], 
               n_factors=params['n_factors'],
               verbose=False)
evaluator.add_algorithm(svd_tuned, "SVD - Tuned")

# Add untuned SVD
svd_untuned = SVD(verbose=False)
evaluator.add_algorithm(svd_untuned, "SVD - Untuned")

# Add random recommendations as baseline
random_rec = NormalPredictor()
evaluator.add_algorithm(random_rec, "Random")

### Evaluation results

In [ ]:
results = evaluator.evaluate(top_n_metrics=True)

results.to_df()

### Sample Recommendations

Generate and display some sample recommendations using the evaluated algorithms.

In [ ]:
samples = evaluator.sample_top_n_recs(uid=85)

for algorithm, recs in samples.items():
    print(f"{algorithm}")
    movie_names = lens.get_movie_names(recs)
    for movie_name in movie_names:
        print(f"  {movie_name}")